In [1]:
# LOAD PACKAGES

import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import rioxarray as rxr
import geopandas as gpd
import xarray as xr
import pandas as pd
import glob
import os
import fnmatch
import math
from math import e
from osgeo import gdal 
print("packages loaded")

packages loaded


In [2]:
# SET DIRECTORIES

# Set basin directory
dnbr_basins = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/output/'
print(sorted(os.listdir(dnbr_basins)))

# Set x_value directory
x_values = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/x_values.csv'
print(x_values)

# Set output directory
out = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/'

# Create dataframe 
df = pd.read_csv(x_values, delimiter = ',')

['12042_dnbr.tif', '12057_dnbr.tif', '12518_dnbr.tif', '12524_dnbr.tif', '12527_dnbr.tif', '12616_dnbr.tif', '12860_dnbr.tif', '12997_dnbr.tif', '13281_dnbr.tif', '17346_dnbr.tif', '17347_dnbr.tif', '17987_dnbr.tif', '19165_dnbr.tif', '19167_dnbr.tif', '19381_dnbr.tif', '19384_dnbr.tif', '19450_dnbr.tif', '19451_dnbr.tif', '19510_dnbr.tif', '19512_dnbr.tif', '19629_dnbr.tif', '19632_dnbr.tif', '19647_dnbr.tif', '19649_dnbr.tif', '19777_dnbr.tif', '20029_dnbr.tif', '20031_dnbr.tif', '20253_dnbr.tif', '20254_dnbr.tif', '20303_dnbr.tif', '20308_dnbr.tif', '20384_dnbr.tif', '20386_dnbr.tif', '20499_dnbr.tif', '20504_dnbr.tif', '20650_dnbr.tif', '20729_dnbr.tif', '20734_dnbr.tif', '21009_dnbr.tif', '21078_dnbr.tif', '21083_dnbr.tif', '21224_dnbr.tif', '21500_dnbr.tif', '21513_dnbr.tif', '21634_dnbr.tif', '22075_dnbr.tif', '22668_dnbr.tif']
/mnt/nfs/lss/meerdink/home/skzebarth/masters/basin_data/x_values.csv


# This block tests specific shapefile x2 values, use if somethin ain't lookin right pardner.
test = '/mnt/nfs/lss/meerdink/home/skzebarth/masters/output/22668_dnbr.tif'
shp = rxr.open_rasterio(test)
sts = np.mean(shp)
print(sts)

In [3]:
# GRAB MEAN VALUE FROM EACH BASIN

# Set empty list for .tif mean, the .item() call grabs the xarrayvalue
myarr = []
for filename in sorted(os.listdir(dnbr_basins)):
    f = os.path.join(dnbr_basins, filename)
    if filename.endswith('.tif'):
        
        #read in tif file
        basin = rxr.open_rasterio(f, masked=True)
        #print(basin)
        b_name = os.path.splitext(filename)[0]
        #print(b_name)
        sts = np.mean(basin)
        value = sts.item()
        #print(value)
        myarr.append(value)
       
#print(myarr)       
df.insert(loc=4, column='s2b_x2', value=myarr)
print(df)
#df.to_csv(out + 'combined_x.csv', index=False)

    basin_id        x1        x2        x3    s2b_x2
0      12042  0.197747  0.233633  0.130000  0.228622
1      12057  0.280069  0.474403  0.130000  0.483068
2      12518  0.166060  0.214940  0.250382  0.215482
3      12524  0.394477  0.466995  0.290000  0.459457
4      12527  0.321767  0.352798  0.290000  0.368283
5      12616  0.074116  0.077151  0.256383  0.093811
6      12860  0.431564  0.413134  0.290000  0.387043
7      12997  0.479127  0.437978  0.290000  0.439869
8      13281  0.126046  0.231041  0.130000  0.227571
9      17346  0.560555  0.476775  0.131622  0.486930
10     17347  0.688192  0.616294  0.141628  0.620807
11     17987  0.550802  0.600549  0.130000  0.597654
12     19165  0.644476  0.475917  0.135718  0.478883
13     19167  0.237853  0.506045  0.130285  0.473715
14     19381  0.144583  0.534886  0.130000  0.507904
15     19384  0.683425  0.542329  0.139364  0.545505
16     19450  0.112769  0.424504  0.130000  0.394791
17     19451  0.243529  0.491567  0.130000  0.

In [4]:
# THOMAS FIRE PRELIMINARY HAZARD ASSESSMENT LINK FUNCTION CALCULATION

# Coefficients for Southern California
b = -3.63
b_1 = 0.41
b_2 = 0.67
b_3 = 0.7

# Rainfall for design storm (12mm, 16mm, 20mm, 24mm, 28mm 32mm, 36mm, 40mm)
## Come back at a later date and make this section iterative as well
#R = 3
#R = 4
#R = 5
R = 6
#R = 7
#R = 8
#R = 9
#R = 10

# Link function set up - empty lists are used to append values from the dataframe calculations
link_x = []
Prob = []
for index, row in df.iterrows():
    x_1 = row['x1']
    #print(x_1)
    x_2 = row['s2b_x2']
    #print(x_2)
    x_3 = row['x3']
    #print(x_3)
    x =  b + (b_1 * x_1 * R) + (b_2 * x_2 * R) + (b_3 * x_3 * R)
    link_x.append(x)

df.insert(loc=5, column='link_x', value=link_x)
print(df)



    basin_id        x1        x2        x3    s2b_x2    link_x
0      12042  0.197747  0.233633  0.130000  0.228622 -1.678483
1      12057  0.280069  0.474403  0.130000  0.483068 -0.453098
2      12518  0.166060  0.214940  0.250382  0.215482 -1.303650
3      12524  0.394477  0.466995  0.290000  0.459457  0.405430
4      12527  0.321767  0.352798  0.290000  0.368283 -0.139955
5      12616  0.074116  0.077151  0.256383  0.093811 -1.993747
6      12860  0.431564  0.413134  0.290000  0.387043  0.205560
7      12997  0.479127  0.437978  0.290000  0.439869  0.534927
8      13281  0.126046  0.231041  0.130000  0.227571 -1.859090
9      17346  0.560555  0.476775  0.131622  0.486930  0.259235
10     17347  0.688192  0.616294  0.141628  0.620807  1.153433
11     17987  0.550802  0.600549  0.130000  0.597654  0.673541
12     19165  0.644476  0.475917  0.135718  0.478883  0.450537
13     19167  0.237853  0.506045  0.130285  0.473715 -0.593352
14     19381  0.144583  0.534886  0.130000  0.507904 -0

In [5]:
# THOMAS FIRE PRELIMINARY HAZARD ASSESSMENT PROBABLITY CALCULATION

# Iterate over link_x rows for debris flow probability value
for index, row in df.iterrows():
    link = row['link_x']
    P = (e ** link) / (1 + e** link)
    Prob.append(P)
    
# Insert values into the dataframe and output results to a .csv    
df.insert(loc=6, column='Probability', value=Prob)
print(df)
df.to_csv(out + 'sentinel_2b_thomas_fire_pha.csv', index=False)

    basin_id        x1        x2        x3    s2b_x2    link_x  Probability
0      12042  0.197747  0.233633  0.130000  0.228622 -1.678483     0.157296
1      12057  0.280069  0.474403  0.130000  0.483068 -0.453098     0.388624
2      12518  0.166060  0.214940  0.250382  0.215482 -1.303650     0.213551
3      12524  0.394477  0.466995  0.290000  0.459457  0.405430     0.599992
4      12527  0.321767  0.352798  0.290000  0.368283 -0.139955     0.465068
5      12616  0.074116  0.077151  0.256383  0.093811 -1.993747     0.119861
6      12860  0.431564  0.413134  0.290000  0.387043  0.205560     0.551210
7      12997  0.479127  0.437978  0.290000  0.439869  0.534927     0.630632
8      13281  0.126046  0.231041  0.130000  0.227571 -1.859090     0.134809
9      17346  0.560555  0.476775  0.131622  0.486930  0.259235     0.564448
10     17347  0.688192  0.616294  0.141628  0.620807  1.153433     0.760137
11     17987  0.550802  0.600549  0.130000  0.597654  0.673541     0.662296
12     19165